In [1]:
import pandas as pd
import numpy as np
from time import time
from collections import Counter
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# data_dir = '/home/ec2-user/SCB_CODE/cam_code/data_180112/'

ld_predict = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/late_days_pred_predict_180101.csv')
ld_predict.head()

,Unnamed: 0,doc_id,latedays_predict,latedays_prob_0,latedays_prob_1,latedays_prob_2
0,0,113NT3SXHF-19069,0,1.000,0.000,0.0
1,1,113NT3SXHF-2564,0,0.980,0.020,0.0
2,2,113NT3SXHF-5029,0,0.995,0.005,0.0
3,3,113XBHSBFT-10433,0,0.980,0.020,0.0
4,4,113XBHSBFT-10599,0,0.905,0.095,0.0


In [3]:
len(ld_predict)

530146

In [4]:
dr_predict = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/dr_predict_180116.csv')
print(len(dr_predict))
dr_predict.head()

530146


,Unnamed: 0,doc_id,dilution_predict,dilution_predict_0,dilution_prob_1,dilution_prob_2
0,0,113NT3SXHF-19069,0,0.788285,0.177737,0.033978
1,1,113NT3SXHF-2564,0,0.721605,0.194411,0.083983
2,2,113NT3SXHF-5029,0,0.733595,0.184587,0.081818
3,3,113XBHSBFT-10433,2,0.317518,0.215679,0.466803
4,4,113XBHSBFT-10599,2,0.272545,0.218451,0.509004


In [64]:
all_prod = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/data_split_flag_180116_dr.csv')
print(len(all_prod))
all_prod.head()

530146


,level_0,index_x,Unnamed: 0.1,cty_code,doc_id,supplier_id,buyer_id,doc_type_code,deal_id,fin_flag,...,client_3m_term,client_3m_late_days,client_3m_isLate,obs_weight_yr,doc_date_TS,test_flag_x,test_flag_y,test_split,buyer_centric_flag,dilution_amt
0,0,61867,335680,SP,113NT3SXHF-19069,100010823,100004692.0,INV,TF540B0037,Y,...,0.000000,0.000000,0.00000,0.166667,2008-11-03,NaN,NaN,train,1,0.000000
1,1,41315,25138,SG,113NT3SXHF-2564,100004586,100015181.0,INV,TF540S0646,Y,...,105.000000,0.000000,0.00000,0.166667,2008-11-27,NaN,NaN,train,0,-1098.634391
2,2,41316,25139,SG,113NT3SXHF-5029,100004586,100015182.0,INV,TF540S0648,Y,...,100.000000,0.000000,0.00000,0.166667,2008-11-27,NaN,NaN,train,0,-1352.165404
3,3,41655,25140,SG,113XBHSBFT-10433,100004672,100004535.0,INV,TF540S0001,Y,...,80.009709,1.378641,0.23301,0.166667,2009-02-02,NaN,NaN,train,0,13891.830000
4,4,41653,25142,SG,113XBHSBFT-10599,100004672,100002504.0,INV,TF540S0001,Y,...,77.493333,1.053333,0.20000,0.166667,2009-01-21,NaN,NaN,train,0,7921.540000


In [6]:
# ld_predict.drop(columns=['Unnamed: 0'], inplace = True)
# dr_predict.drop(columns=['Unnamed: 0'], inplace = True)
all_prod.drop(columns=list(all_prod.columns)[:3], inplace=True)
ld_predict.head(), dr_predict.head(), all_prod.head()

(   Unnamed: 0            doc_id  latedays_predict  latedays_prob_0  \
 0           0  113NT3SXHF-19069                 0            1.000   
 1           1   113NT3SXHF-2564                 0            0.980   
 2           2   113NT3SXHF-5029                 0            0.995   
 3           3  113XBHSBFT-10433                 0            0.980   
 4           4  113XBHSBFT-10599                 0            0.905   
 
    latedays_prob_1  latedays_prob_2  
 0            0.000              0.0  
 1            0.020              0.0  
 2            0.005              0.0  
 3            0.020              0.0  
 4            0.095              0.0  ,
    Unnamed: 0            doc_id  dilution_predict  dilution_predict_0  \
 0           0  113NT3SXHF-19069                 0            0.788285   
 1           1   113NT3SXHF-2564                 0            0.721605   
 2           2   113NT3SXHF-5029                 0            0.733595   
 3           3  113XBHSBFT-10433         

In [7]:
data = all_prod.merge(ld_predict, how = 'left', on = 'doc_id').merge(dr_predict, how = 'left', on = 'doc_id')
data.head()

,cty_code,doc_id,supplier_id,buyer_id,doc_type_code,deal_id,fin_flag,rec_id,doc_date,doc_ccy_code,...,Unnamed: 0_x,latedays_predict,latedays_prob_0,latedays_prob_1,latedays_prob_2,Unnamed: 0_y,dilution_predict,dilution_predict_0,dilution_prob_1,dilution_prob_2
0,SP,113NT3SXHF-19069,100010823,100004692.0,INV,TF540B0037,Y,4005372,2008-11-03,USD,...,0,0,1.000,0.000,0.0,0,0,0.788285,0.177737,0.033978
1,SG,113NT3SXHF-2564,100004586,100015181.0,INV,TF540S0646,Y,3995308,2008-11-27,SGD,...,1,0,0.980,0.020,0.0,1,0,0.721605,0.194411,0.083983
2,SG,113NT3SXHF-5029,100004586,100015182.0,INV,TF540S0648,Y,3996605,2008-11-27,SGD,...,2,0,0.995,0.005,0.0,2,0,0.733595,0.184587,0.081818
3,SG,113XBHSBFT-10433,100004672,100004535.0,INV,TF540S0001,Y,12498608,2009-02-02,USD,...,3,0,0.980,0.020,0.0,3,2,0.317518,0.215679,0.466803
4,SG,113XBHSBFT-10599,100004672,100002504.0,INV,TF540S0001,Y,8435131,2009-01-21,USD,...,4,0,0.905,0.095,0.0,4,2,0.272545,0.218451,0.509004


In [8]:
tab_file = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/all_prod_pred_rsrv_hk_sample_only (1).csv', nrows=10)

In [11]:
tab_columns=list(tab_file.columns)

# tab_columns = ['Unnamed: 0.1', 'invoice_id', 'supplier_id', 'buyer_id', 'invoice_date', 'invoice_amt', 
#                'invoice_due_date', 'financing_amt', 'financing_approved', 'financing_date', 'maturity_date', 
#                'financing_payment_amt', 'financing_payment_date', 'cty_code_x', 'lcg_crdt_grd_cd_val', 'term_days', 
#                 'islate', 'isLate_pred', 'isLate_pred_prob_0', 'isLate_pred_prob_1', 
#                'deal_id', 'final_dilution', 'dilution_bucket', 'dilution_predict', 'dilution_predict_0', 
#                'dilution_prob_1', 'dilution_prob_2', 'product', 'cust_id', 'fin_tenor_start_date', 'fin_tenor', 
#                'grace_period', 'late_days_f']

len(tab_columns)

43

In [12]:
tab_columns

['Unnamed: 0',
 'Unnamed: 0.1',
 'invoice_id',
 'supplier_id',
 'buyer_id',
 'invoice_date',
 'invoice_amt',
 'invoice_due_date',
 'financing_amt',
 'financing_approved',
 'financing_date',
 'maturity_date',
 'financing_payment_amt',
 'financing_payment_date',
 'otp_party_id',
 'cty_code_x',
 'lcg_crdt_grd_cd_val',
 'term_days',
 'finance_days',
 'finance_due_days',
 'islate',
 'isLate_pred',
 'isLate_pred_prob_0',
 'isLate_pred_prob_1',
 'deal_id',
 'final_dilution',
 'dilution_bucket',
 'dilution_predict',
 'dilution_predict_0',
 'dilution_prob_1',
 'dilution_prob_2',
 'product',
 'cust_id',
 'doc_ccy_code',
 'fin_ccy_code',
 'fin_tenor_start_date',
 'fin_tenor',
 'doc_ccy_amt',
 'from_seq_no',
 'grace_period',
 'pymt_ccy_code',
 'to_seq_no',
 'late_days_f']

In [13]:
all_prod = data



['CG', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'buyer_id',
       'cty_code', 'cust_id', 'deal_id', 'doc_ccy_amt', 'doc_ccy_code',
       'doc_date', 'doc_id', 'due_date', 'fin_ccy_code', 'fin_tenor',
       'fin_tenor_start_date', 'financing_amount',
       'financing_payment_amount', 'from_seq_no', 'grace_period',
       'invoice_amt_usd', 'maturity_date', 'product', 'pymt_ccy_code',
       'pymt_value_date', 'start_date', 'supplier_id', 'to_seq_no',
       'x_rate']


In [14]:
all_prod_col = list(all_prod.columns)

not_in_all_prod =[x for x in tab_columns if x not in all_prod_col]
not_in_all_prod

['Unnamed: 0',
 'Unnamed: 0.1',
 'invoice_id',
 'invoice_date',
 'invoice_amt',
 'invoice_due_date',
 'financing_amt',
 'financing_approved',
 'financing_date',
 'financing_payment_amt',
 'financing_payment_date',
 'otp_party_id',
 'cty_code_x',
 'lcg_crdt_grd_cd_val',
 'term_days',
 'islate',
 'isLate_pred',
 'isLate_pred_prob_0',
 'isLate_pred_prob_1',
 'final_dilution',
 'dilution_bucket',
 'cust_id',
 'pymt_ccy_code',
 'late_days_f']

In [15]:
rename_col_names = {
#     'Unnamed: 0',
#  'Unnamed: 0.1',
 'doc_id':'invoice_id',
 'doc_date':'invoice_date',
 'doc_usd_amt':'invoice_amt',
 'due_date':'invoice_due_date',
 'fin_usd_amt':'financing_amt',
 'fin_flag':'financing_approved',
 'fin_date':'financing_date',
 'pymt_usd_amt':'financing_payment_amt',
 'pymt_date':'financing_payment_date',
#  'otp_party_id',
 'cty_code':'cty_code_x',
 'CG':'lcg_crdt_grd_cd_val',
 'tenor':'term_days',
 'isLate':'islate',
 'latedays_predict':'isLate_pred',
 'latedays_prob_0':'isLate_pred_prob_0',
 'latedays_prob_1':'isLate_pred_prob_1', 
 'dr':'final_dilution',
 'dr_bucket':'dilution_bucket',
 'client':'cust_id',
#  'pymt_ccy_code',
 'late_days':'late_days_f'
}

In [16]:
all_prod.rename(rename_col_names, axis = 1, inplace=True)
all_prod_col = list(all_prod.columns)
not_in_all_prod =[x for x in tab_columns if x not in all_prod_col]
not_in_all_prod

['Unnamed: 0', 'Unnamed: 0.1', 'otp_party_id', 'pymt_ccy_code']

In [17]:
all_prod['Unnamed: 0'] = all_prod.index
all_prod['Unnamed: 0.1'] = all_prod.index
all_prod['otp_party_id'] = all_prod.cust_id
all_prod['pymt_ccy_code'] = all_prod.fin_ccy_code

all_prod_col = list(all_prod.columns)
not_in_all_prod =[x for x in tab_columns if x not in all_prod_col]
not_in_all_prod

[]

In [18]:
all_prod.rename({"latedays_prob_2":"late_prob_2"}, axis = 1, inplace=True)

In [19]:
all_prod_tab = all_prod[['Unnamed: 0.1', 'invoice_id', 'supplier_id', 'buyer_id', 'invoice_date', 'invoice_amt', 
               'invoice_due_date', 'financing_amt', 'financing_approved', 'financing_date', 'maturity_date', 
               'financing_payment_amt', 'financing_payment_date', 'cty_code_x', 'lcg_crdt_grd_cd_val', 'term_days', 
                'islate', 'isLate_pred', 'isLate_pred_prob_0', 'isLate_pred_prob_1', 'late_prob_2',
               'deal_id', 'final_dilution', 'dilution_bucket', 'dilution_predict', 'dilution_predict_0', 
               'dilution_prob_1', 'dilution_prob_2', 'product', 'cust_id', 'fin_tenor_start_date', 'fin_tenor', 
               'grace_period', 'late_days_f','late_days_bucket', 'product_variant_code', 'test_split']]

all_prod_tab.head()

,Unnamed: 0.1,invoice_id,supplier_id,buyer_id,invoice_date,invoice_amt,invoice_due_date,financing_amt,financing_approved,financing_date,...,dilution_prob_2,product,cust_id,fin_tenor_start_date,fin_tenor,grace_period,late_days_f,late_days_bucket,product_variant_code,test_split
0,0,113NT3SXHF-19069,100010823,100004692.0,2008-11-03,67890.63000,2009-03-03,67890.630000,Y,2008-12-05,...,0.033978,IFNB,100010823,2008-11-03 00:00:00,120.0,0,0,0,IFNB02,train
1,1,113NT3SXHF-2564,100004586,100015181.0,2008-11-27,51747.40272,2009-02-25,52846.037111,Y,2008-12-05,...,0.083983,IFNS,100004586,2008-11-27 00:00:00,90.0,0,0,0,IFNS01,train
2,2,113NT3SXHF-5029,100004586,100015182.0,2008-11-27,63689.11104,2009-02-25,65041.276444,Y,2008-12-05,...,0.081818,IFNS,100004586,2008-11-27 00:00:00,90.0,0,0,0,IFNS01,train
3,3,113XBHSBFT-10433,100004672,100004535.0,2009-02-02,92612.18000,2009-04-30,78720.350000,Y,2009-02-06,...,0.466803,IFNS,100004672,2009-02-02 00:00:00,87.0,0,0,0,IFNS02,train
4,4,113XBHSBFT-10599,100004672,100002504.0,2009-01-21,52810.25000,2009-04-21,44888.710000,Y,2009-02-09,...,0.509004,IFNS,100004672,2009-01-21 00:00:00,90.0,0,0,0,IFNS02,train


In [20]:
all_prod_tab.to_csv('/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/all_prod_pred.csv')

In [22]:
### islate
islate = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/drive-download-20180105T234953Z-001/Islatedistribution.csv', nrows=10)
islate_col = islate.columns

not_in_islate = [x for x in islate_col if x not in list(all_prod_tab.columns)]
not_in_islate

['Unnamed: 0']

In [23]:
islate_col=islate_col[1:]
islate_tab = all_prod[islate_col]
islate_tab.head()

,cust_id,invoice_id,invoice_date,invoice_amt,financing_date,financing_amt,cty_code_x,late_days_f,product,islate,final_dilution,dilution_bucket,dilution_predict
0,100010823,113NT3SXHF-19069,2008-11-03,67890.63000,2008-12-05,67890.630000,SP,0,IFNB,0,0.000000,0,0
1,100004586,113NT3SXHF-2564,2008-11-27,51747.40272,2008-12-05,52846.037111,SG,0,IFNS,0,-0.021231,0,0
2,100004586,113NT3SXHF-5029,2008-11-27,63689.11104,2008-12-05,65041.276444,SG,0,IFNS,0,-0.021231,0,0
3,100004672,113XBHSBFT-10433,2009-02-02,92612.18000,2009-02-06,78720.350000,SG,0,IFNS,0,0.150000,2,2
4,100004672,113XBHSBFT-10599,2009-01-21,52810.25000,2009-02-09,44888.710000,SG,0,IFNS,0,0.150000,2,2


In [103]:
islate_tab['client_ID'] = islate_tab.cust_id

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [79]:
len(all_prod_tab), len(islate_tab)

(530145, 530146)

In [104]:
islate_tab.to_csv('/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/Islatedistribution.csv',index=False)

In [25]:
### all_scores

all_scores = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/drive-download-20180105T234953Z-001/all_score_agg.csv', nrows=10)

In [26]:
all_scores.head()

,Unnamed: 0,features,dr_weights,islate_weights
0,0,CG,0.040469,0.023821
1,1,CG_no_data,0.001512,0.000000
2,2,buy_annual_inv_cnt,0.001703,0.005496
3,3,buy_ave_annual_fin_amt,0.000667,0.008170
4,4,buy_ave_fin_amt,0.011411,0.042085


In [27]:
dr_scores = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/DR_logreg_scores.csv')

In [43]:
ld_scores = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/DR_logreg_scores_latedays_(1).csv')
print(dr_scores.head())
print(ld_scores.head())

          index  bucket_0  bucket_1  bucket_2  dr_weights
0    Unnamed: 0  0.000000  1.000000  2.000000    2.000000
1         tenor -3.703612  3.696502  0.007111    3.703612
2  grace_period -4.333200  1.907162  2.426038    4.333200
3     fin_tenor  5.448782 -6.244758  0.795976    6.244758
4   doc_usd_amt  1.190781 -0.704749 -0.486032    1.190781
   Unnamed: 0      tenor  grace_period  fin_tenor  doc_usd_amt  \
0           0  13.321308    -17.145780  -7.954594    -0.576831   
1           1  -5.613025     -9.180717   5.147229     0.422243   
2           2  -7.708283     26.326497   2.807364     0.154587   

   buyer_supplier_yrs  buyer_supplier_past_inv_cnt  \
0                 0.0                     1.915780   
1                 0.0                     0.138069   
2                 0.0                    -2.053849   

   buyer_supplier_annual_inv_cnt  buyer_supplier_ave_fin_amt  \
0                       1.915780                    -0.84104   
1                       0.138069          

In [44]:
# dr_scores = dr_scores.transpose()
ld_scores = ld_scores.transpose()
dr_scores.head, ld_scores.head()

(<bound method DataFrame.head of                                  index   bucket_0  bucket_1   bucket_2  \
 0                           Unnamed: 0   0.000000  1.000000   2.000000   
 1                                tenor  -3.703612  3.696502   0.007111   
 2                         grace_period  -4.333200  1.907162   2.426038   
 3                            fin_tenor   5.448782 -6.244758   0.795976   
 4                          doc_usd_amt   1.190781 -0.704749  -0.486032   
 5                   buyer_supplier_yrs   0.000000  0.000000   0.000000   
 6          buyer_supplier_past_inv_cnt   1.806672  4.010367  -5.817038   
 7        buyer_supplier_annual_inv_cnt   1.806672  4.010367  -5.817038   
 8           buyer_supplier_ave_fin_amt   0.319253 -0.212936  -0.106317   
 9    buyer_supplier_ave_annual_fin_amt   2.093843  0.575408  -2.669251   
 10          buyer_supplier_std_fin_amt   2.332744 -1.520721  -0.812023   
 11            buyer_supplier_late_days  -0.605188 -0.349486   0.954

In [31]:
dr_scores.columns = ['bucket_0', 'bucket_1', 'bucket_2']

In [41]:
dr_scores_arr = np.asarray(dr_scores)
dr_weights = []
for i in range(len(dr_scores_arr)):
    dr_weights.append(np.max([abs(x) for x in dr_scores_arr[i][1:]]))
    
dr_scores['dr_weights'] = dr_weights
dr_scores.describe()

,bucket_0,bucket_1,bucket_2,dr_weights
count,137.000000,137.000000,137.000000,137.000000
mean,1.008492,-0.175799,-0.810795,2.468056
std,3.922863,1.566246,3.361753,3.380796
min,-29.819311,-6.244758,-6.664211,0.000000
25%,0.108131,-0.734837,-1.834987,1.025235
50%,1.340355,-0.276619,-0.996922,1.600263
75%,2.209804,0.165025,-0.094477,2.840733
max,7.363655,10.179903,24.897330,29.819311


In [45]:
ld_scores.columns = ['bucket_0', 'bucket_1', 'bucket_2']
ld_scores_arr = np.asarray(ld_scores)
islate_weights = []
for i in range(len(ld_scores_arr)):
    islate_weights.append(np.max([abs(x) for x in ld_scores_arr[i][1:]]))
    
ld_scores['islate_weights'] = islate_weights
ld_scores.describe()

,bucket_0,bucket_1,bucket_2,islate_weights
count,181.000000,181.000000,181.000000,181.000000
mean,0.151993,-0.058660,-0.076758,1.062049
std,1.948461,1.303403,2.395731,2.248360
min,-17.145780,-9.180717,-7.708283,0.000000
25%,0.000000,-0.202773,-0.589131,0.000000
50%,0.000000,0.000000,-0.006760,0.516400
75%,0.591258,0.299060,0.000000,1.246534
max,13.321308,5.147229,26.326497,26.326497


In [46]:
dr_scores.columns

Index([u'index', u'bucket_0', u'bucket_1', u'bucket_2', u'dr_weights'], dtype='object')

In [48]:
dr_scores.head()

,index,bucket_0,bucket_1,bucket_2,dr_weights
0,Unnamed: 0,0.000000,1.000000,2.000000,2.000000
1,tenor,-3.703612,3.696502,0.007111,3.703612
2,grace_period,-4.333200,1.907162,2.426038,4.333200
3,fin_tenor,5.448782,-6.244758,0.795976,6.244758
4,doc_usd_amt,1.190781,-0.704749,-0.486032,1.190781


In [52]:
dr_scores['features'] = dr_scores['index']
dr_scores.head()

,index,bucket_0,bucket_1,bucket_2,dr_weights,features
0,Unnamed: 0,0.000000,1.000000,2.000000,2.000000,Unnamed: 0
1,tenor,-3.703612,3.696502,0.007111,3.703612,tenor
2,grace_period,-4.333200,1.907162,2.426038,4.333200,grace_period
3,fin_tenor,5.448782,-6.244758,0.795976,6.244758,fin_tenor
4,doc_usd_amt,1.190781,-0.704749,-0.486032,1.190781,doc_usd_amt


In [54]:
ld_scores['features'] = ld_scores.index
ld_scores.head()

,bucket_0,bucket_1,bucket_2,islate_weights,features
Unnamed: 0,0.000000,1.000000,2.000000,2.000000,Unnamed: 0
tenor,13.321308,-5.613025,-7.708283,7.708283,tenor
grace_period,-17.145780,-9.180717,26.326497,26.326497,grace_period
fin_tenor,-7.954594,5.147229,2.807364,5.147229,fin_tenor
doc_usd_amt,-0.576831,0.422243,0.154587,0.422243,doc_usd_amt


In [55]:
all_scores_tab = dr_scores[['features', 'dr_weights']].merge(ld_scores[['features', 'islate_weights']], how = 'inner', on = 'features')
all_scores_tab.head()

,features,dr_weights,islate_weights
0,Unnamed: 0,2.000000,2.000000
1,tenor,3.703612,7.708283
2,grace_period,4.333200,26.326497
3,fin_tenor,6.244758,5.147229
4,doc_usd_amt,1.190781,0.422243


In [57]:
all_scores_tab.drop([0], axis = 0, inplace=True)
all_scores_tab.head()

,features,dr_weights,islate_weights
1,tenor,3.703612,7.708283
2,grace_period,4.333200,26.326497
3,fin_tenor,6.244758,5.147229
4,doc_usd_amt,1.190781,0.422243
5,buyer_supplier_yrs,0.000000,0.000000


In [58]:
all_scores_tab.to_csv('/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/all_score_agg.csv')

### all_scores_agg

In [62]:
all_scores_agg2 = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/drive-download-20180105T234953Z-001/all_score_agg_2.csv', nrows=5)
all_scores_agg2.head()

,Invoice Number,features,dr_weight,islate_weight
0,DN7Z1P3VWR-22947,CG,0.22,0.33
1,RG5351XR95-10996,CG,0.71,0.84
2,YB7N5JMX63-38256,CG,-0.66,-0.86
3,2D8KK6XXZ9-75987,CG,0.75,0.60
4,289WDND3FL-13099,CG,-0.01,-0.62


In [60]:
scaled = pd.read_csv('/home/ec2-user/SCB_CODE/cam_code/data_180112/scaled_features_180116.csv')
scaled.head()

,tenor,grace_period,fin_tenor,doc_usd_amt,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,...,CG_5B,CG_6A,CG_6B,CG_7A,CG_7B,CG_8A,CG_8B,CG_9A,CG_9B,CG_NO_CG
0,0.064004,0.0,0.013289,0.000290,0.0,0.000016,0.000016,0.000206,0.000009,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.050386,0.0,0.009967,0.000221,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.050386,0.0,0.009967,0.000272,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.049024,0.0,0.009635,0.000396,0.0,0.001769,0.001769,0.000082,0.000409,0.000437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.050386,0.0,0.009967,0.000226,0.0,0.000243,0.000243,0.000050,0.000034,0.000232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [65]:
len(scaled), len(all_prod)

(530146, 530146)

In [66]:
scaled['doc_id'] = all_prod.doc_id
scaled.head()

,tenor,grace_period,fin_tenor,doc_usd_amt,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,...,CG_6A,CG_6B,CG_7A,CG_7B,CG_8A,CG_8B,CG_9A,CG_9B,CG_NO_CG,doc_id
0,0.064004,0.0,0.013289,0.000290,0.0,0.000016,0.000016,0.000206,0.000009,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113NT3SXHF-19069
1,0.050386,0.0,0.009967,0.000221,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113NT3SXHF-2564
2,0.050386,0.0,0.009967,0.000272,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113NT3SXHF-5029
3,0.049024,0.0,0.009635,0.000396,0.0,0.001769,0.001769,0.000082,0.000409,0.000437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113XBHSBFT-10433
4,0.050386,0.0,0.009967,0.000226,0.0,0.000243,0.000243,0.000050,0.000034,0.000232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,113XBHSBFT-10599


In [86]:
IFNS_scores = scaled[scaled.product_IFNS==1]

In [87]:
IFNS_scores = scaled[scaled.product_IFNS==1]
IFNS_scores.describe()

,tenor,grace_period,fin_tenor,doc_usd_amt,buyer_supplier_yrs,buyer_supplier_past_inv_cnt,buyer_supplier_annual_inv_cnt,buyer_supplier_ave_fin_amt,buyer_supplier_ave_annual_fin_amt,buyer_supplier_std_fin_amt,...,CG_5B,CG_6A,CG_6B,CG_7A,CG_7B,CG_8A,CG_8B,CG_9A,CG_9B,CG_NO_CG
count,28510.000000,28510.000000,28510.000000,2.851000e+04,28510.0,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000,...,28510.0,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000,28510.000000
mean,0.050896,0.001238,0.006206,1.282990e-04,0.0,0.018918,0.018918,0.000119,0.000733,0.000219,...,0.0,0.000105,0.000070,0.000035,0.163697,0.065907,0.078709,0.014802,0.040407,0.587759
std,0.018158,0.007296,0.009381,1.459932e-03,0.0,0.026446,0.026446,0.001215,0.001319,0.001378,...,0.0,0.010258,0.008375,0.005922,0.370007,0.248123,0.269289,0.120761,0.196915,0.492247
min,0.009532,0.000000,0.000000,3.479369e-11,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.041307,0.000000,0.003322,2.986735e-06,0.0,0.000730,0.000730,0.000004,0.000060,0.000005,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.063096,0.000000,0.003322,6.885246e-06,0.0,0.005243,0.005243,0.000017,0.000293,0.000044,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.064004,0.000000,0.009967,3.521386e-05,0.0,0.028339,0.028339,0.000039,0.000823,0.000117,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,0.109396,0.120000,1.000000,8.345026e-02,0.0,0.103585,0.103585,0.054162,0.029620,0.035562,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [68]:
IFNS_scoresm = IFNS_scores.melt(id_vars='doc_id',var_name='features', value_name='scaled')
IFNS_scoresm.head()

,doc_id,features,scaled
0,113NT3SXHF-2564,tenor,0.050386
1,113NT3SXHF-5029,tenor,0.050386
2,113XBHSBFT-10433,tenor,0.049024
3,113XBHSBFT-10599,tenor,0.050386
4,113XBHSBFT-3164,tenor,0.036768


In [88]:
len(IFNS_scoresm)

107848

In [69]:
IFNS_scoresm = IFNS_scoresm.merge(dr_predict[['doc_id', 'dilution_predict']], how = 'inner', on='doc_id').merge(
    ld_predict[['doc_id', 'latedays_predict']], how = 'inner', on='doc_id')
IFNS_scoresm.head()

,doc_id,features,scaled,dilution_predict,latedays_predict
0,113NT3SXHF-2564,tenor,0.050386,0,0
1,113NT3SXHF-2564,grace_period,0.000000,0,0
2,113NT3SXHF-2564,fin_tenor,0.009967,0,0
3,113NT3SXHF-2564,doc_usd_amt,0.000221,0,0
4,113NT3SXHF-2564,buyer_supplier_yrs,0.000000,0,0


In [89]:
len(IFNS_scoresm)

107848

In [75]:
#melt coeffs by bucket
dr_melt = dr_scores[['features', 'bucket_0', 'bucket_1','bucket_2']].melt(id_vars='features', value_name = 'dr_coeffs', var_name = 'dilution_predict')
dr_melt['dilution_predict'] = [int(x[-1]) for x in dr_melt.dilution_predict]
dr_melt.head()

,features,dilution_predict,dr_coeffs
0,Unnamed: 0,0,0.000000
1,tenor,0,-3.703612
2,grace_period,0,-4.333200
3,fin_tenor,0,5.448782
4,doc_usd_amt,0,1.190781


In [76]:
ld_melt = ld_scores[['features', 'bucket_0', 'bucket_1','bucket_2']].melt(id_vars='features', value_name = 'ld_coeffs', var_name = 'latedays_predict')
ld_melt['latedays_predict'] = [int(x[-1]) for x in ld_melt.latedays_predict]
ld_melt.head()

,features,latedays_predict,ld_coeffs
0,Unnamed: 0,0,0.000000
1,tenor,0,13.321308
2,grace_period,0,-17.145780
3,fin_tenor,0,-7.954594
4,doc_usd_amt,0,-0.576831


In [91]:
IFNS_merge = IFNS_scoresm.merge(dr_melt, how='inner', on=['features','dilution_predict'])
len(IFNS_merge)

107848

In [92]:
IFNS_merge2 = IFNS_merge.merge(ld_melt, how='inner', on=['features','latedays_predict'])
len(IFNS_merge2)

107848

In [77]:
#merge coeffs with scaled
IFNS_merge = IFNS_scoresm.merge(dr_melt, how='inner', on=['features','dilution_predict']
                               ).merge(ld_melt, how='inner', on=['features','latedays_predict'])
IFNS_merge.head()

,doc_id,features,scaled,dilution_predict,latedays_predict,dr_coeffs,ld_coeffs
0,113NT3SXHF-2564,tenor,0.050386,0,0,-3.703612,13.321308
1,113NT3SXHF-5029,tenor,0.050386,0,0,-3.703612,13.321308
2,113XBHSBFT-3164,tenor,0.036768,0,0,-3.703612,13.321308
3,11BQ5RQKH7-35688,tenor,0.015433,0,0,-3.703612,13.321308
4,11BQ5RQKH7-35689,tenor,0.015433,0,0,-3.703612,13.321308


In [90]:
len(IFNS_merge)

3877360

In [94]:
IFNS_merge2['dr_weight'] = IFNS_merge2.scaled*IFNS_merge2.dr_coeffs
IFNS_merge2['islate_weight'] = IFNS_merge2.scaled*IFNS_merge2.ld_coeffs
IFNS_merge2.rename({'doc_id':'Invoice Number'}, axis=1, inplace=True)
IFNS_final = IFNS_merge2[['Invoice Number', 'features','dr_weight', 'islate_weight']]
print(len(IFNS_final))
IFNS_final.head()

107848


,Invoice Number,features,dr_weight,islate_weight
0,18HR18CH85-84112,tenor,-0.085740,-0.129943
1,18HR18CH85-84358,tenor,-0.085740,-0.129943
2,18HR18CH85-84510,tenor,-0.094145,-0.142682
3,18HR18CH85-84801,tenor,-0.110957,-0.168161
4,18HR18CH85-85011,tenor,-0.085740,-0.129943


In [123]:
IFNS_final.describe()

,dr_weight,islate_weight
count,107848.000000,107848.000000
mean,0.025889,0.012086
std,0.953056,0.196964
min,-8.825870,-2.767505
25%,0.000000,-0.000000
50%,0.000000,-0.000000
75%,0.000000,0.000000
max,4.501173,6.113410


In [95]:
IFNS_final.to_csv('/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/all_score_agg_2_IFNS.csv', index=False)

In [100]:
#other prods
prod = 'OTHERS'
IFNS_scores = scaled[(scaled.product_IFNB==0)
                     &(scaled.product_IFNS==0)
                     &(scaled.product_TRDS==0)
                     &(scaled.product_RSRV==0)]
print(len(IFNS_scores))
IFNS_scoresm = IFNS_scores.melt(id_vars='doc_id',var_name='features', value_name='scaled')
# IFNS_merge = IFNS_scoresm.merge(dr_melt, how='inner', on=['features','dilution_predict'])
print(len(IFNS_scoresm))
IFNS_scoresm = IFNS_scoresm.merge(dr_predict[['doc_id', 'dilution_predict']], how = 'inner', on='doc_id').merge(
    ld_predict[['doc_id', 'latedays_predict']], how = 'inner', on='doc_id')
IFNS_merge = IFNS_scoresm.merge(dr_melt, how='inner', on=['features','dilution_predict']
                               ).merge(ld_melt, how='inner', on=['features','latedays_predict'])
print(len(IFNS_merge)), IFNS_merge.head()
IFNS_merge['dr_weight'] = IFNS_merge.scaled*IFNS_merge.dr_coeffs
IFNS_merge['islate_weight'] = IFNS_merge.scaled*IFNS_merge.ld_coeffs
IFNS_merge.rename({'doc_id':'Invoice Number'}, axis=1, inplace=True)
IFNS_final = IFNS_merge[['Invoice Number', 'features','dr_weight', 'islate_weight']]
print(prod, len(IFNS_final))
IFNS_final.to_csv('/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/all_score_agg_2_OTHERS.csv', index=False)

2047
278392
278392              doc_id features    scaled  dilution_predict  latedays_predict  \
0  128MKP8P4K-69052    tenor  0.019973                 0                 0   
1  12H6L617DQ-96924    tenor  0.024966                 0                 0   
2  159LYKNN67-99736    tenor  0.021788                 0                 0   
3  16HDK6HBXZ-90435    tenor  0.022696                 0                 0   
4  17FFJMCS8Q-40844    tenor  0.019973                 0                 0   

   dr_coeffs  ld_coeffs  
0  -3.703612  13.321308  
1  -3.703612  13.321308  
2  -3.703612  13.321308  
3  -3.703612  13.321308  
4  -3.703612  13.321308  
('OTHERS', 278392)


In [101]:
products = ['IFNB', 'RSRV', 'TRDS']
for prod in products:
    product_col = ''.join(['product_',prod])
    IFNS_scores = scaled[scaled[product_col]==1]
    print(len(IFNS_scores))
    IFNS_scoresm = IFNS_scores.melt(id_vars='doc_id',var_name='features', value_name='scaled')
    
    print(len(IFNS_scoresm))
    IFNS_scoresm = IFNS_scoresm.merge(dr_predict[['doc_id', 'dilution_predict']], how = 'inner', on='doc_id').merge(
        ld_predict[['doc_id', 'latedays_predict']], how = 'inner', on='doc_id')
    IFNS_merge = IFNS_scoresm.merge(dr_melt, how='inner', on=['features','dilution_predict']
                                   ).merge(ld_melt, how='inner', on=['features','latedays_predict'])
    print(len(IFNS_merge)), IFNS_merge.head()
    IFNS_merge['dr_weight'] = IFNS_merge.scaled*IFNS_merge.dr_coeffs
    IFNS_merge['islate_weight'] = IFNS_merge.scaled*IFNS_merge.ld_coeffs
    IFNS_merge.rename({'doc_id':'Invoice Number'}, axis=1, inplace=True)
    IFNS_final = IFNS_merge[['Invoice Number', 'features','dr_weight', 'islate_weight']]
    print(prod, len(IFNS_final))
    IFNS_final.to_csv("".join(['/home/ec2-user/SCB_CODE/cam_code/Tableau_poc_180118/all_score_agg_2_',prod,'.csv']), index=False)
    

296428
40314208
40314208              doc_id features    scaled  dilution_predict  latedays_predict  \
0  113NT3SXHF-19069    tenor  0.064004                 0                 0   
1   113XBHSBFT-7629    tenor  0.077621                 0                 0   
2  11B2H3JBLN-22107    tenor  0.037222                 0                 0   
3  11BQ5RQKH7-11336    tenor  0.023150                 0                 0   
4  11BQ5RQKH7-12684    tenor  0.023150                 0                 0   

   dr_coeffs  ld_coeffs  
0  -3.703612  13.321308  
1  -3.703612  13.321308  
2  -3.703612  13.321308  
3  -3.703612  13.321308  
4  -3.703612  13.321308  
('IFNB', 40314208)
202368
27522048
27522048              doc_id features    scaled  dilution_predict  latedays_predict  \
0  116CPXSPYV-31590    tenor  0.037222                 2                 0   
1  11H9PJYBMS-18218    tenor  0.064004                 2                 0   
2  128KKVW8FQ-76116    tenor  0.023150                 2                

### summary table shuye for POC presentation

In [105]:
len(all_prod)

530146

In [120]:
metrics = {'buyer_id':'nunique', 'supplier_id':'nunique', 'doc_id':'nunique', 
           'doc_date':['min','max'], 'doc_usd_amt':'sum', 'fin_usd_amt':'sum','pymt_usd_amt':'sum','cty_code':'nunique' }


In [121]:
all_prod.groupby('product').agg(metrics)

cty_code supplier_id    doc_date              pymt_usd_amt  \
         nunique     nunique         min         max           sum   
product                                                              
BATB           2           4  2015-03-06  2016-10-19  1.690865e+07   
ECIF           1           1  2012-10-11  2015-07-21  2.704845e+07   
ECPR           1           2  2014-05-27  2014-12-26  2.594796e+06   
IFNB          16        1488  2008-06-16  2017-11-30  9.715340e+09   
IFNS          12         144  2001-04-25  2017-11-30  8.258810e+08   
LBDS           2           2  2013-02-14  2015-12-03  1.185413e+07   
LRRP           3          31  2011-09-20  2017-10-13  6.032243e+10   
PSHP           1           1  2017-03-24  2017-03-24  5.695415e+04   
RSRV          22         337  2008-01-24  2017-09-25  2.687287e+09   
TRDS           7          10  2016-02-23  2017-09-12  1.502215e+08   

          doc_usd_amt  doc_id   fin_usd_amt buyer_id  
                  sum nunique           sum  nunique  
product                                               
BATB     1.690865e+07       9  1.690865e+07        3  
ECIF     2.704845e+07     182  2.704845e+07       16  
ECPR     4.733704e+06       3  2.594796e+06        2  
IFNB     9.844940e+09  296428  9.715340e+09      738  
IFNS     8.562129e+08   28510  8.258810e+08      866  
LBDS     1.194908e+07     177  1.185413e+07        2  
LRRP     6.033418e+10    1675  6.032243e+10      113  
PSHP     5.724346e+04       1  5.695415e+04        1  
RSRV     2.807690e+09  202368  2.687469e+09     1242  
TRDS     1.578259e+08     793  1.504004e+08       53

In [108]:
doc_date = [pd.Timestamp(x) for x in all_prod.doc_date]

In [110]:
all_prod['doc_date_TS1'] = doc_date
all_prod['yr_1516'] = all_prod[(all_prod.doc_date_TS>=pd.datetime('2015-01-01'))]
#                                &(all_prod.doc_date_TS<pd.datetime('2017-01-01'))]

TypeError: an integer is required

In [113]:
doc_yr = [x.year for x in doc_date]
Counter(doc_yr)

Counter({2001: 1,
         2008: 5824,
         2009: 5167,
         2010: 9919,
         2011: 22686,
         2012: 31389,
         2013: 79201,
         2014: 90505,
         2015: 99831,
         2016: 110936,
         2017: 74687})

In [115]:
all_prod['doc_yr'] = doc_yr
all_prod_1516 = all_prod[(all_prod.doc_yr>=2015)&(all_prod.doc_yr<=2016)]
all_prod_1516.groupby('product').agg(metrics)

deal_id bucket_61   fin_usd_amt supplier_id bucket_31_60        dr  \
        nunique       sum           sum     nunique          sum      mean   
product                                                                      
BATB          9         0  1.690865e+07           4            0  0.000000   
ECIF         11         0  9.851853e+05           1            0  0.000000   
IFNB      30440      5501  3.091228e+09         252          658  0.007796   
IFNS       2022         0  3.647488e+08          44            6  0.012440   
LBDS          3         0  2.799899e+04           1            0  0.000000   
LRRP        370         0  9.371490e+09          13            0  0.000000   
RSRV       1712       127  6.870476e+08         145          159  0.012572   
TRDS          8         0  5.986152e+07           7            0  0.000000   

        bucket_1_30  pymt_usd_amt    fin_date             buyer_id  doc_id  \
                sum           sum         min         max  nunique nunique   
product                                                                      
BATB              0  1.690865e+07  2015-03-06  2016-10-31        3       9   
ECIF              0  9.851853e+05  2015-05-06  2015-07-21        4      11   
IFNB            862  3.091228e+09  2015-01-02  2017-03-08      128  108224   
IFNS            369  3.647488e+08  2015-01-06  2017-01-20      136    8589   
LBDS              2  2.799899e+04  2015-03-30  2015-12-03        1       3   
LRRP              1  9.371490e+09  2015-01-07  2016-12-30       53     376   
RSRV          66279  6.870476e+08  2015-01-06  2017-05-03      555   93529   
TRDS             18  5.986152e+07  2016-02-26  2017-06-05       11      26   

          doc_usd_amt  
                  sum  
product                
BATB     1.690865e+07  
ECIF     9.851853e+05  
IFNB     3.101632e+09  
IFNS     3.694192e+08  
LBDS     2.799899e+04  
LRRP     9.371490e+09  
RSRV     7.196797e+08  
TRDS     5.986152e+07

In [122]:
all_prod_1516.groupby('product').agg(metrics)

cty_code supplier_id    doc_date              pymt_usd_amt  \
         nunique     nunique         min         max           sum   
product                                                              
BATB           2           4  2015-03-06  2016-10-19  1.690865e+07   
ECIF           1           1  2015-05-06  2015-07-21  9.851853e+05   
IFNB           7         252  2015-01-01  2016-12-31  3.091228e+09   
IFNS           5          44  2015-01-01  2016-12-31  3.647488e+08   
LBDS           1           1  2015-03-30  2015-12-03  2.799899e+04   
LRRP           2          13  2015-01-05  2016-12-29  9.371490e+09   
RSRV          17         145  2015-01-02  2016-12-31  6.870476e+08   
TRDS           5           7  2016-02-23  2016-12-29  5.986152e+07   

          doc_usd_amt  doc_id   fin_usd_amt buyer_id  
                  sum nunique           sum  nunique  
product                                               
BATB     1.690865e+07       9  1.690865e+07        3  
ECIF     9.851853e+05      11  9.851853e+05        4  
IFNB     3.101632e+09  108224  3.091228e+09      128  
IFNS     3.694192e+08    8589  3.647488e+08      136  
LBDS     2.799899e+04       3  2.799899e+04        1  
LRRP     9.371490e+09     376  9.371490e+09       53  
RSRV     7.196797e+08   93529  6.870476e+08      555  
TRDS     5.986152e+07      26  5.986152e+07       11

In [124]:
len(all_prod)

530146

In [125]:
len(pd.unique(all_prod.buyer_id))

3010

In [126]:
len(pd.unique(all_prod.supplier_id))

1999

In [127]:
def count_unique(df_series):
    return(len(pd.unique(df_series)))

count_unique(all_prod.supplier_id)

1999

In [128]:
all_prod.groupby(['product']).agg({'buyer_id':count_unique, 'supplier_id':count_unique})

,supplier_id,buyer_id
product,,
BATB,4,3.0
ECIF,1,16.0
ECPR,2,2.0
IFNB,1488,738.0
IFNS,144,866.0
LBDS,2,2.0
LRRP,31,113.0
PSHP,1,1.0
RSRV,337,1242.0


In [129]:
count_unique(all_prod.client)

1999